In [1]:
import scipy.io
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
from Encoding import load
dataset_name = "2Sphere"
par_data, bc = load(dataset_name,data_type="par_data"), load(dataset_name,data_type="bc")

In [3]:
from Encoding import ProjectPointsToCylinder, ProjectPointsToHyperplane, ConstructTopology
par = par_data[0][0]
bc_t = bc[0]
super_topology = ConstructTopology(par,bc_t,4)

In [4]:
from Encoding import TopologySlice, CheckForWallContact
from Evaluation import NumpyGroupby


from Encoding import TopologyFromPlausibleTopology

topology = TopologyFromPlausibleTopology(super_topology,par,bc_t)

In [5]:
topology.shape, super_topology.shape

((5, 2), (13, 2))

In [6]:
super_topology,topology

(array([[ 0,  1],
        [ 0, -1],
        [ 1, -1],
        [ 0, -2],
        [ 1, -2],
        [ 0, -3],
        [ 1, -3],
        [ 0, -4],
        [ 1, -4],
        [ 0, -5],
        [ 1, -5],
        [ 0, -6],
        [ 1, -6]]),
 array([[ 0,  1],
        [ 1, -4],
        [ 1, -2],
        [ 0, -2],
        [ 0, -1]]))

In [ ]:
from Encoding import BCEncoding

P_virtual, top_new = BCEncoding(par_data[0][0],top,bc_t)
P_virtual, top_new

In [ ]:
point_on_wall = ProjectPointsToCylinder(particles,cyl)
#point_on_wall = ProjectPointsToHyperplane(particles,wall)


In [ ]:
from Plotting import Plot3DVectors
#fig = plt.figure()
#ax = fig.add_subplot(projection='3d')
#Plot3DVectors(ax,axis_projection,s)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = P_virtual[:,:3]
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
ax.scatter(x,y,z)

In [ ]:
vector = point_on_wall
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='b')
vector = particles
x = vector[:,0]
y = vector[:,1]
z = vector[:,2]
plt.plot(y,z,'o',color='r')